# From Prev.
#Rearranged code
#Used larger training set
#Match Similar words manually (due to limited size of categories [hopefully :P])
#text = re.sub("[^ก-๛]", '', text) 47.34% with TFIDF
#text = re.sub("[^ก-ไ๐-๙]", '', text) 48.44% with TFIDF
#text = re.sub("[^ก-ไ๐-๙‘’“”]", '', text) 48.99% with TFIDF
#used news_content instead of news_label, pretty fast, 61.83%

In [79]:
import pandas as pd
from urllib.parse import unquote

df = pd.read_json('data-1.json',encoding='utf-8')
#df.drop(['date','body','author','url'],axis=1,inplace=True) #for test_2.json
#df.drop(['id','news_content','date','url'],axis=1,inplace=True)

In [80]:
CAT= {
    #category: change to this
    'ข่าวบันเทิง':"entertainment",
    'regional':'local',
    'sport':'sports',
    'ข่าวกีฬา':'sports',
    'ข่าวอาชญากรรม':'crime',
    'khrongkhaidaasapparod':'ข่าวโครงข่ายตาสับปะรด',
    'around-the-world-news':'foreign',
    'abroad':'foreign',
    'ข่าวต่างประเทศ':'foreign',
    'ข่าวไลฟ์สไตล์':'lifestyle',
    'analysis-today-politics':'politics',
    'politic':'politics',
    'ข่าวเศรษฐกิจ':'economics',
    'economy':'economics',
    'economic':'economics',
    'uncategorized':'others',
    'auto':'car-vehicle',
    'thai-soccer':'thaifootball',
    'ข่าวพระราชสำนัก':'royalnews',
    'it':'tech'
}

In [81]:
def conv_cate(category):
    result = CAT.get(category,-1)
    if result!=-1:
        return result
    else :
        #in case want to update dictionary
        #CAT[category] = category
        return category

In [82]:
df['category'] = df['category'].apply(unquote).apply(conv_cate)

In [83]:
#tags cleaning
#df[df['category']=='1293427']
#df.drop([292],inplace=True)
df['category'].unique()
len(df['category'].unique())

53

In [84]:
df.isnull().sum()
#df.dropna().reset_index(drop=True)

id              0
news_topic      0
news_content    0
date            0
author          0
url             0
category        0
dtype: int64

In [85]:
import numpy as np
category = np.array(df['category'])
tags, freq = np.unique(category,return_counts=True)
freqList = list(zip(tags,freq))
freqList.sort(key=lambda elem:elem[1]) #sort by frequency
freqList[20:]

[('car-vehicle', 5),
 ('stock-monitor', 5),
 ('thaifootball', 6),
 ('covid-19', 7),
 ('index.php', 7),
 ('ข่าวโครงข่ายตาสับปะรด', 9),
 ('life', 10),
 ('women', 14),
 ('ข่าวในประเทศ', 14),
 ('bangkok', 15),
 ('bbc-thai', 18),
 ('trcheerthai', 18),
 ('special-stories', 20),
 ('society', 21),
 ('lifestyle', 22),
 ('tech', 22),
 ('breaking-news', 23),
 ('ข่าวด่วน', 23),
 ('pr-news', 28),
 ('news', 29),
 ('education', 30),
 ('others', 31),
 ('around-thailand', 32),
 ('business', 34),
 ('eurofootball', 78),
 ('crime', 80),
 ('economics', 96),
 ('article', 98),
 ('entertainment', 129),
 ('sports', 257),
 ('politics', 261),
 ('foreign', 338),
 ('local', 357)]

In [86]:
df.head(1)

,id,news_topic,news_content,date,author,url,category
0,1,"""นกเอี้ยง จันทิมา""โดนหนุ่มจับก้น เจ้าตัวตบสวนถ...",นกเอี้ยง จันทิมา นักร้องสาว โพสต์เฟสบุ๊กส่วนตั...,2019-06-29 18:30:44,ch3,http://news.ch3thailand.com/entertainment/98347,entertainment


In [191]:

X = np.array(df['news_content'])
y = np.array(df['category'])

# Cleaning text


In [192]:
re.sub("[^ก-ไ๐-๙‘’“”]", '', 'ฟหกดเ้่าสาสงๆไำพัีนรนบลผปแอิืมทมใฝๅ/_ภถุึคึre.sub("[^ก-ไ๐-๙‘’“”]"ฟหกดเ้่าสาสงๆไำพัีนรนบลผปแอิืมคตจจตช!!!')
#after searching thai ascii

'ฟหกดเาสาสงไำพัีนรนบลผปแอิืมทมใฝภถุึคึกไ๐๙‘’“”ฟหกดเาสาสงไำพัีนรนบลผปแอิืมคตจจตช'

In [193]:
import re
#prepare each sentence
def text_prepare(text):
    #text = re.sub("[/(){}\[\]\|@,;\n\t:\'\-!,'์'‘’“”]", '', text)
    #text = re.sub("[/\xa0]", '', text)
    #text = re.sub("[0-9]", '', text)
    text = re.sub("[^ก-ไ๐-๙‘’“”]", '', text)
    return text

In [194]:
#execute prepare
X = np.array([text_prepare(body) for body in X])

# Prepare train and test set

In [195]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [196]:
type(X_train)

numpy.ndarray

In [197]:
len(X_test)

545

## Check tag distribution

In [198]:
#not well distributed
tags, freq = np.unique(y_train,return_counts=True)
list(zip(tags,freq))

[('advertorial', 4),
 ('agriculture', 5),
 ('around-thailand', 21),
 ('article', 79),
 ('bangkok', 12),
 ('bbc-thai', 15),
 ('beauty', 2),
 ('boxing-mma', 1),
 ('breaking-news', 12),
 ('business', 25),
 ('car-vehicle', 3),
 ('covid-19', 6),
 ('crime', 65),
 ('drama', 1),
 ('economics', 73),
 ('education', 22),
 ('entertainment', 94),
 ('eurofootball', 59),
 ('food', 2),
 ('foreign', 250),
 ('hot-topics', 1),
 ('house-condo', 2),
 ('in-the-news', 1),
 ('index.php', 6),
 ('life', 9),
 ('lifestyle', 15),
 ('live-from-the-scene', 1),
 ('local', 269),
 ('monitor-news', 2),
 ('news', 25),
 ('online-social', 2),
 ('others', 22),
 ('politics', 193),
 ('pr-news', 21),
 ('society', 14),
 ('special-stories', 15),
 ('sports', 196),
 ('stock-monitor', 4),
 ('tech', 18),
 ('thaifootball', 4),
 ('travel', 1),
 ('trcheerthai', 11),
 ('volleyball', 2),
 ('women', 11),
 ('worldcup', 2),
 ('ข่าวด่วน', 21),
 ('ข่าวโครงข่ายตาสับปะรด', 4),
 ('ข่าวในประเทศ', 9),
 ('ดวง', 1),
 ('หวย', 2)]

# Tokenize thai words

In [199]:
from pythainlp.tokenize import word_tokenize

In [200]:

#tokenize function
def Thaitokenize(text):
    """
    turn a Thai sentence into Thai words with meaning
    """
    text = ''.join(text.split('"')) ##handled in regex??
    text = word_tokenize(text , engine = 'pyicu')
    return text

In [201]:
#try mapping function
#Error
#list(map(Thaitokenize,X_train))

In [202]:
Thaitokenize('แสนสิริสร้างUN Global Standards of Conduct for Business\xa0')

['แสน',
 'สิริ',
 'สร้าง',
 'UN',
 ' ',
 'Global',
 ' ',
 'Standards',
 ' ',
 'of',
 ' ',
 'Conduct',
 ' ',
 'for',
 ' ',
 'Business',
 '\xa0']

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'pyicu')

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'newmm')

In [203]:
#words to account in the bag
words_counts = {}

for i in X_train: #each i is a topic sentence
    wordlist = Thaitokenize(i)
    for j in wordlist:
        words_counts[j] = words_counts.get(j,0) + 1
#words_counts = 238

WORDS_TO_INDEX = dict(zip(words_counts.keys(),list(x for x in range(len(words_counts)))))

In [204]:
len(words_counts)

16456

# Create bag of words

In [205]:
#create a bag of words for a sentence
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a sentence
        words_to_index: words to count in the bag 
        e.g.
        words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
        
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    
    #tokenize Thai words
    words = Thaitokenize(text)
    
    for i in words:
      if(words_to_index.get(i,-1)!=-1) :
        result_vector[words_to_index[i]]=+1
    return result_vector

In [206]:
#X_train_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_train])

# TFIDF

In [207]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [208]:
#X_train_tfidf = vectorizer.fit_transform(X_train)
#print(vectorizer.get_feature_names())
#print(X.shape)

In [209]:
vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.9, min_df = 5)
#vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.9, min_df = 0)
#max_df=0.8, min_df=5 not much difference
#vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.8, min_df = 10)
X_train_tfidf = vectorizer.fit_transform(X_train)
print(vectorizer.get_feature_names())
print(X_train_tfidf.shape)

['ก', 'กก', 'กขอให', 'กค', 'กช', 'กซ', 'กฎ', 'กฎระเบียบ', 'กฎหมาย', 'กด', 'กดดัน', 'กต', 'กตัญญู', 'กติกา', 'กถา', 'กทม', 'กน', 'กนก', 'กนิษฐา', 'กบ', 'กบฏ', 'กป', 'กพ', 'กพนี', 'กพบ', 'กภจว', 'กม', 'กย', 'กยนี', 'กร', 'กรก', 'กรกฎาคม', 'กรง', 'กรณี', 'กรณีพิเศษ', 'กรณีศึกษา', 'กรด', 'กรม', 'กรมการ', 'กรมการพัฒนาชุมชน', 'กรมอุตุนิยมวิทยา', 'กรร', 'กรรม', 'กรรมการ', 'กรรมาธิการ', 'กรวม', 'กรอ', 'กรอก', 'กรอง', 'กรอบ', 'กระ', 'กระจก', 'กระจาย', 'กระจายเสียง', 'กระชับ', 'กระชาก', 'กระดาษ', 'กระดูก', 'กระทบ', 'กระทรวง', 'กระทรวงกลาโหม', 'กระทรวงการคลัง', 'กระทรวงคมนาคม', 'กระทรวงมหาดไทย', 'กระทรวงยุติธรรม', 'กระทรวงศึกษาธิการ', 'กระทรวงสาธารณสุข', 'กระทรวงอุตสาหกรรม', 'กระทรวงเกษตรฯ', 'กระทำ', 'กระทำความผิด', 'กระทิง', 'กระบวน', 'กระบวนการ', 'กระบอก', 'กระบะ', 'กระสุน', 'กระแทก', 'กระแส', 'กระแสไฟ', 'กระโดด', 'กรัก', 'กรับ', 'กรัม', 'กราด', 'กราน', 'กราบ', 'กราย', 'กราว', 'กรี', 'กรีก', 'กรีด', 'กรีน', 'กรุ', 'กรุง', 'กรุงศรีอยุธยา', 'กรุงเทพ', 'กรุงเทพมหานคร', 'กรุงเทพฯ', 'กรุงไทย', 'กรุณ

In [210]:
#[word for word in WORDS_TO_INDEX]
#type(WORDS_TO_INDEX.keys())
#X_train_tfidf

##ARRAY <-> Sparse matrix much faster
#X_train_tfidf.toarray() 
#sparse matrix to array
#from scipy import sparse
#sparse.csr_matrix(X_train_tfidf.toarray()) 
#array to sparse matrix

In [211]:
#for illustration & checking
featurelist = vectorizer.get_feature_names()
TFIDF = pd.DataFrame(X_train_tfidf.toarray(),columns = featurelist)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
TFIDF


ก   กก  กขอให   กค   กช   กซ   กฎ  กฎระเบียบ  กฎหมาย        กด  \
0     0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
1     0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
2     0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
3     0.030733  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.054144   
4     0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
...        ...  ...    ...  ...  ...  ...  ...        ...     ...       ...   
1630  0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
1631  0.081848  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
1632  0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
1633  0.015705  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   
1634  0.000000  0.0    0.0  0.0  0.0  0.0  0.0        0.0     0.0  0.000000   

      กดดัน   กต  กตัญญู  กติกา  กถา       กทม   กน  กนก  กนิษฐา   กบ  กบฏ  \
0       0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
1       0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
2       0.0  0.0     0.0    0.0  0.0  0.056273  0.0  0.0     0.0  0.0  0.0   
3       0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
4       0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
...     ...  ...     ...    ...  ...       ...  ...  ...     ...  ...  ...   
1630    0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
1631    0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
1632    0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
1633    0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   
1634    0.0  0.0     0.0    0.0  0.0  0.000000  0.0  0.0     0.0  0.0  0.0   

       กป   กพ  กพนี  กพบ  กภจว   กม   กย  กยนี   กร  กรก  กรกฎาคม       กรง  \
0     0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
1     0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
2     0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
3     0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
4     0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.347841   
...   ...  ...   ...  ...   ...  ...  ...   ...  ...  ...      ...       ...   
1630  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
1631  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
1632  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
1633  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   
1634  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0      0.0  0.000000   

          กรณี  กรณีพิเศษ  กรณีศึกษา  กรด  กรม  กรมการ  กรมการพัฒนาชุมชน  \
0     0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
1     0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
2     0.057382        0.0        0.0  0.0  0.0     0.0               0.0   
3     0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
4     0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
...        ...        ...        ...  ...  ...     ...               ...   
1630  0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
1631  0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
1632  0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
1633  0.000000        0.0        0.0  0.0  0.0     0.0               0.0   
1634  0.000000        0.0        0.0  0.0  0.0     0.0               0.0   

      กรมอุตุนิยมวิทยา  กรร  กรรม  กรรมการ  กรรมาธิการ  กรวม  กรอ  กรอก  กรอง  \
0                  0.0  0.0   0.0      0.0         0.0   0.0  0.0   0.0   0.0   
1                  0.0  0.0   0.0      0.0         0.0   0.0  0.0   0.0   0.0   
2         

In [212]:
X_train_tfidf

<1635x5104 sparse matrix of type '<class 'numpy.float64'>'
	with 272937 stored elements in Compressed Sparse Row format>

# turn y values to 1s,0s

In [213]:
# from sklearn.preprocessing import MultiLabelBinarizer
# this is for list of tags

In [214]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
#lb.fit_transform([1, 2, 6, 4, 2]) 
#fit & transform in one step
#fit = figure out different classes and turn into necessary binary formats
#transform = transform given data into binary formats

In [215]:
y_train_lb = lb.fit_transform(y_train)


In [216]:
lb.classes_.size

50

In [217]:
#for illustration & checking
Y_LB = pd.DataFrame(y_train_lb,columns = lb.classes_)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
Y_LB.iloc[10:20]
#y_train.iloc[10:20]

,advertorial,agriculture,around-thailand,article,bangkok,bbc-thai,beauty,boxing-mma,breaking-news,business,car-vehicle,covid-19,crime,drama,economics,education,entertainment,eurofootball,food,foreign,hot-topics,house-condo,in-the-news,index.php,life,lifestyle,live-from-the-scene,local,monitor-news,news,online-social,others,politics,pr-news,society,special-stories,sports,stock-monitor,tech,thaifootball,travel,trcheerthai,volleyball,women,worldcup,ข่าวด่วน,ข่าวโครงข่ายตาสับปะรด,ข่าวในประเทศ,ดวง,หวย
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [218]:
y_train_lb.shape

(1635, 50)

# Train model to predict tags

In [219]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#example
X = np.array([
     [10, 10],
     [8, 10],
     [-5, 5.5],
     [-5.4, 5.5],
     [-20, -20],
     [-15, -20]])
y = np.array([0, 0, 1, 1, 2, 2])
clf = OneVsRestClassifier(SVC()).fit(X, y)
clf.predict([[-19, -20], [9, 9], [-5, 5]])

array([2, 0, 1])

In [220]:
clf = OneVsRestClassifier(SVC()).fit(X_train_tfidf, y_train)

In [221]:
y_train

array(['lifestyle', 'entertainment', 'local', ..., 'economics', 'crime',
       'bangkok'], dtype=object)

In [222]:
#check the predictions, models applied correctly? Yes
y = clf.predict(X_train_tfidf)
print(np.concatenate((y.reshape(len(y),1), y_train.reshape(len(y_train),1)),1))

[['lifestyle' 'lifestyle']
 ['entertainment' 'entertainment']
 ['local' 'local']
 ...
 ['economics' 'economics']
 ['crime' 'crime']
 ['bangkok' 'bangkok']]


In [223]:
X_test = np.array([text_prepare(title) for title in X_test])
#X_test_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_test])
X_test_tfidf = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_tfidf)
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#print(np.concatenate(y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1),1))

[['foreign' 'economics']
 ['local' 'crime']
 ['foreign' 'foreign']
 ...
 ['local' 'local']
 ['politics' 'politics']
 ['foreign' 'foreign']]


In [224]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 7 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


0.618348623853211

In [225]:
test = text_prepare('“อนุทิน” ขอ ผู้ร้อง เปิดชื่อ อักษรย่อ \"พ\" คนหักหัวคิวโรงแรม 40%')
#test = np.array([my_bag_of_words(test, WORDS_TO_INDEX, len(words_counts))])
test = vectorizer.transform([test])
clf.predict(test)

array(['politics'], dtype='<U21')